# Isaac Lab 4WD Robot Training on Google Colab

This notebook sets up Isaac Lab on Google Colab and trains a 4WD robot with LiDAR-based obstacle avoidance.

**Requirements:**
- Colab Runtime: GPU (L4 or A100 recommended)
- Runtime shape: Standard or High-RAM

**Note:** First run will take ~15-20 minutes for installation.

## 1. Check GPU Availability

In [ ]:
!nvidia-smi

## 2. Install Isaac Lab Dependencies

Isaac Lab requires Isaac Sim, which is large (~20GB). We'll use the containerized version.

In [ ]:
# Install system dependencies
!apt-get update && apt-get install -y \
    git \
    git-lfs \
    wget \
    libglib2.0-0 \
    libsm6 \
    libxext6 \
    libxrender-dev \
    libgomp1 \
    libglu1-mesa

## 3. Clone Isaac Lab Repository

In [ ]:
import os

# /contentに移動してからクローン
%cd /content

# Clone Isaac Lab if not already present
if not os.path.exists('/content/IsaacLab'):
    print("Cloning Isaac Lab...")
    !git clone https://github.com/isaac-sim/IsaacLab.git /content/IsaacLab
    print("✓ Isaac Lab cloned successfully")
else:
    print("✓ Isaac Lab already exists")

# IsaacLabディレクトリに移動
%cd /content/IsaacLab
print(f"Current directory: {os.getcwd()}")

## 4. Install Isaac Lab

This uses the installation script provided by NVIDIA.

In [ ]:
# Run the installation script (headless mode for Colab)
!./isaaclab.sh --install

## 5. Upload Project Files

Upload your project files from local machine:
- `assets/robots/4wd_vehicle.urdf`
- `assets/config/vehicle_config.yaml`
- `envs/isaac_4wd_env.py`
- `scripts/train.py`

In [ ]:
# Create project structure
!mkdir -p /content/isaac_4wd_rl/assets/robots
!mkdir -p /content/isaac_4wd_rl/assets/config
!mkdir -p /content/isaac_4wd_rl/envs
!mkdir -p /content/isaac_4wd_rl/scripts

# Upload files using Colab's file upload
from google.colab import files

print("Upload 4wd_vehicle.urdf:")
uploaded = files.upload()
for filename in uploaded.keys():
    !mv {filename} /content/isaac_4wd_rl/assets/robots/

print("\nUpload vehicle_config.yaml:")
uploaded = files.upload()
for filename in uploaded.keys():
    !mv {filename} /content/isaac_4wd_rl/assets/config/

print("\nUpload isaac_4wd_env.py:")
uploaded = files.upload()
for filename in uploaded.keys():
    !mv {filename} /content/isaac_4wd_rl/envs/

print("\nUpload train.py:")
uploaded = files.upload()
for filename in uploaded.keys():
    !mv {filename} /content/isaac_4wd_rl/scripts/

## 6. Alternative: Clone from GitHub

If you've pushed your project to GitHub, you can clone it directly:

In [ ]:
# /contentに移動
%cd /content

# 既存のプロジェクトを削除
import shutil
if os.path.exists('/content/isaac_4wd_rl'):
    print("Removing existing project...")
    shutil.rmtree('/content/isaac_4wd_rl')

# Clone from GitHub (推奨)
print("Cloning project from GitHub...")
!git clone https://github.com/n01boy/isaac_4wd_rl.git /content/isaac_4wd_rl

# プロジェクトディレクトリに移動
%cd /content/isaac_4wd_rl

# 依存関係をインストール
!pip install -q -r requirements.txt

print("✓ Project ready")

## 7. Install Additional Python Dependencies

In [ ]:
!pip install pyyaml tensorboard onnx onnxruntime

## 8. Set Up Environment Variables

In [ ]:
import os

# Add Isaac Lab to Python path
os.environ['PYTHONPATH'] = f"/content/IsaacLab:{os.environ.get('PYTHONPATH', '')}"

# Set project path
os.environ['PROJECT_PATH'] = '/content/isaac_4wd_rl'

# Set headless mode (no GUI rendering on Colab)
os.environ['HEADLESS'] = '1'

print("Environment configured successfully!")

## 9. Verify Installation - Test Simple Environment

In [ ]:
# Test Isaac Lab installation
%cd /content/IsaacLab
!python source/standalone/tutorials/00_sim/create_empty.py --headless

## 10. Test 4WD Environment (Single Instance)

In [ ]:
import sys
sys.path.insert(0, '/content/isaac_4wd_rl')

from envs.isaac_4wd_env import FourWDEnvCfg
from omni.isaac.lab.envs import ManagerBasedRLEnv

# Create environment with 1 robot for testing
env_cfg = FourWDEnvCfg()
env_cfg.scene.num_envs = 1

env = ManagerBasedRLEnv(cfg=env_cfg)

print(f"Environment created successfully!")
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

# Test random actions
obs, _ = env.reset()
for i in range(100):
    actions = env.action_space.sample()
    obs, rewards, dones, truncated, info = env.step(actions)
    if i % 20 == 0:
        print(f"Step {i}: Reward = {rewards[0]:.3f}")

env.close()
print("\nTest completed successfully!")

## 11. Start Training with PPO

In [ ]:
%cd /content/isaac_4wd_rl

# Start training (this will run for several hours)
!python scripts/train.py \
    --num_envs 10 \
    --headless \
    --enable_cameras False

## 12. Monitor Training with TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/isaac_4wd_rl/logs

## 13. Export Trained Model to ONNX

In [ ]:
# Export the trained model for Raspberry Pi deployment
!python scripts/export_onnx.py \
    --checkpoint logs/latest_model.pth \
    --output models/4wd_policy.onnx

## 14. Download Trained Model

In [ ]:
from google.colab import files

# Download the ONNX model to your local machine
files.download('/content/isaac_4wd_rl/models/4wd_policy.onnx')

print("Model downloaded! Transfer this to your Raspberry Pi.")

## 15. Save Checkpoint to Google Drive (Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy logs and models to Drive
!cp -r /content/isaac_4wd_rl/logs /content/drive/MyDrive/isaac_4wd_training/
!cp -r /content/isaac_4wd_rl/models /content/drive/MyDrive/isaac_4wd_training/

print("Training data saved to Google Drive!")